In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, grid

In [ ]:

#create the amount raised by percent ahead or behidn opponents and call is amountranksimple (but it's more complex..)

candiates = pd.read_csv(f"../data/candidate_summary_with_labels.csv")

In [160]:
candiates.loc[candiates.duplicated(subset=['CID', 'CycleCands']), :]

,Cycle,RecipID,TotalGiftAmt,TotalGiftAvg,TotalGiftCount,TotalGiftAvg.1,DonorCount,MultiCycleDonorCount,RepeatDonorCount,MaleDonorCount,...,nopacs,republican,democrat,other_party,cand_party,cand_status,run_for_state,run_for_district,presidential_year,label
28,2010,N00000826,813995,825.552738,1167,1.183570,986,347,414,720,...,0,0,1,0,Democrat,Win,NJ,01,0,1
134,2010,N00003572,228913,538.618824,545,1.282353,425,95,134,199,...,0,0,1,0,Democrat,Win,OH,10,0,1
202,2010,N00005906,158966,819.412371,210,1.082474,194,96,103,146,...,0,1,0,0,Republican,Win,TX,14,0,1
836,2010,N00030728,2907,2907.000000,16,16.000000,1,0,1,1,...,0,0,1,0,Democrat,NonIncumbent,CA,33,0,0
2135,2012,N00001279,263619,955.141304,438,1.586957,276,12,103,169,...,0,0,0,1,Other,Lose,FL,20,1,0
2389,2012,N00013808,220445,1281.656977,239,1.389535,172,46,82,129,...,0,1,0,0,Republican,Incumbent,MI,11,1,0
2497,2012,N00027493,5047046,747.157069,10604,1.569800,6755,1418,2587,4765,...,0,1,0,0,Republican,Win,MN,06,1,1
4711,2014,N00034889,21900,509.302326,51,1.186047,43,3,7,32,...,0,1,0,0,Republican,NonIncumbent,MA,S2,0,0
5948,2016,N00030612,48485888,1174.390544,130478,3.160345,41286,3386,22303,27587,...,0,1,0,0,Republican,Win,FL,S2,1,1
5974,2016,N00030836,8421348,714.581926,37761,3.204158,11785,1391,6802,8970,...,0,1,0,0,Republican,Win,KY,S2,1,1


In [148]:


ranks = pd.DataFrame()
simple_ranks = pd.DataFrame()
results=pd.DataFrame()


for y in [2010, 2012, 2014, 2016, 2018]:
    open_seats = candiates.loc[candiates['Cycle'] == y]['DistIDRunFor'].unique()

    for seat in open_seats:

        #get all candidates for this seat
        seat_candidates = pd.DataFrame(candiates.loc[ (candiates['DistIDRunFor'] == seat) & (candiates['Cycle'] == y) ])

        #get the top amount for that seat
        max_amount = seat_candidates['TotalGiftAmt'].max()

        n = len(seat_candidates)

        seat_candidates = seat_candidates.sort_values(by='TotalGiftAmt', ascending=False, ignore_index=True)

        for i in seat_candidates.index:

            amount = seat_candidates['TotalGiftAmt'][i]
            cid = seat_candidates['CID'][i]
            year = seat_candidates['CycleCands'][i]

            rank = pd.DataFrame( [[cid,year, (amount/max_amount)]], columns=['CID','CycleCands', 'AmountRank'])
            ranks = ranks.append(rank)


            simple_rank = None
            if i==0:
                #top amount
                if n > 1:
                    competitor_amount = seat_candidates['TotalGiftAmt'][1]
                    if competitor_amount ==0:
                        if amount>0:
                            simple_rank = pd.DataFrame([[cid, year, 1]], columns=['CID', 'CycleCands', 'AmountRankSimple'])
                        else:
                            simple_rank = pd.DataFrame([[cid,year,  2]], columns=['CID', 'CycleCands', 'AmountRankSimple'])
                        simple_rank = simple_ranks.append(simple_rank)
                        continue

                    percent_diff = abs( competitor_amount - amount ) /competitor_amount
                    if  percent_diff > .05:
                        simple_rank = pd.DataFrame([[cid,year,  1]], columns=['CID', 'CycleCands', 'AmountRankSimple'])

                    else:
                        simple_rank = pd.DataFrame([[cid,year, 2]], columns=['CID', 'CycleCands', 'AmountRankSimple'])

                else:
                    #no competitors
                    simple_rank = pd.DataFrame([[cid,year, 1]], columns=['CID', 'CycleCands', 'AmountRankSimple'])

            else:
                #not top amount
                top_amount = seat_candidates['TotalGiftAmt'][0]
                percent_diff = abs( top_amount - amount ) /amount
                if amount ==0:
                    if top_amount>0:
                        simple_rank = pd.DataFrame([[cid,year, 3]], columns=['CID', 'CycleCands', 'AmountRankSimple'])
                    else:
                        simple_rank = pd.DataFrame([[cid,year, 2]], columns=['CID', 'CycleCands', 'AmountRankSimple'])
                    simple_rank = simple_ranks.append(simple_rank)
                    continue

                if percent_diff <=.05:
                    simple_rank = pd.DataFrame([[cid,year,2]], columns=['CID', 'CycleCands', 'AmountRankSimple'])
                else:
                    simple_rank = pd.DataFrame([[cid,year, 3]], columns=['CID', 'CycleCands', 'AmountRankSimple'])

            if simple_rank.empty:
                raise Exception('Not good')

            simple_ranks = simple_ranks.append(simple_rank)


results = ranks

In [149]:
dup = results.duplicated(subset=['CID', 'CycleCands'])



(9592,)

In [156]:
dup.sum()

11

In [150]:
results.shape

(9592, 3)

In [151]:

results = results.merge(simple_ranks, how='left', on=['CID', 'CycleCands'])

results.shape

(9614, 4)

In [152]:
no_dup = results.drop_duplicates(
  subset = ['CycleCands', 'CID'],
  keep = 'first').reset_index(drop = True)

no_dup.shape

(9581, 4)

In [153]:
no_dup.isnull().sum()

CID                  0
CycleCands           0
AmountRank           0
AmountRankSimple    23
dtype: int64

In [154]:
no_dup = no_dup.fillna(value={'AmountRankSimple':2})

no_dup.isnull().sum()

CID                 0
CycleCands          0
AmountRank          0
AmountRankSimple    0
dtype: int64

In [155]:
no_dup.to_csv(f"../data/amount_rank.csv")